In [1]:
%load_ext autoreload
%autoreload 2
from whr import *

%load_ext pyinstrument

%config InlineBackend.figure_format = 'retina'
# Initialize random number generator
RANDOM_SEED = 8927
rng = np.random.default_rng(RANDOM_SEED)
az.style.use("arviz-darkgrid")

In [2]:
da = PreprocessedData.load()
onlyEvenGames = da.restrictGames(da.games.winnerCount == da.games.loserCount)

using cached data from cache/games_2826287116270d9688585c8305a40ff5.pickle
using cached data from cache/playerNames_d41d8cd98f00b204e9800998ecf8427e.pickle
using cached data from cache/playerDays_c759d8f251d4174b6fe1d8159b20fcc4.pickle


In [3]:
evenModel = createCumsumMatrixModel(onlyEvenGames, separateVirtualGames=True, useExtraPlayerAdjustment=False)
idata = az.InferenceData.from_netcdf('even-games_cumsum-matrix-model-1-year.netcdf')

In [4]:
unevenPlayerDays = evenModel.ratingIxLookup[\
    da.gamePlayers[[]].join(\
        da.games[lambda g: g.winnerCount != g.loserCount][['day']], \
        on='battle_id',how='inner'\
        ).droplevel('battle_id').set_index('day',append=True).index.drop_duplicates()\
    ].sort_index()
unevenPlayerDays

player  day       
1403    2022-09-04        2
3060    2022-06-26        8
        2022-07-03        9
3061    2022-06-26       27
        2022-07-03       28
                      ...  
580786  2023-04-29    66191
        2023-04-30    66192
580815  2023-04-30    66194
580831  2023-04-30    66196
580835  2023-04-30    66197
Length: 34718, dtype: int64

In [5]:
post = idata.posterior
incr : xa.DataArray = post['ratings']
X = incr.isel(playerDay=unevenPlayerDays.values).stack(sample=('chain','draw')).transpose().to_numpy().astype(np.float64)
X.shape

(4000, 34718)

In [6]:
# corr = np.cov(X, rowvar=False) # segfaults, probably because the result would be singular

In [7]:
means = X.mean(axis=0)
Xc = X - means

In [8]:
# Cov = Xc.T @ (Xc  / (X.shape[0]-1))
import scipy.linalg
U, s, Vh = scipy.linalg.svd(Xc, full_matrices=False)

U.shape, s.shape, Vh.shape

# Cov = Xh X = V s² Vh

((4000, 4000), (4000,), (4000, 34718))

In [9]:
(Xc[:,0] @ Xc[:,2])/3999, ((s * Vh[:,0]) @ (s * Vh[:,2]))/3999


(-0.06736020799607169, -0.06736020799607155)

In [ ]:
plt.plot(s.cumsum()/s.sum())

In [16]:
import dtale
namedIndex = unevenPlayerDays.to_frame('ix').join(da.players, on='player').set_index('name',append=True).swaplevel(2,1)

# pc0 = pd.Series(Vh[0], index=namedIndex.index).sort_values()
pcs = pd.DataFrame(Vh[:30].T, index=namedIndex.index)

dtale.show(10000*pcs)


2023-06-10 06:07:15,085 - INFO     - Executing shutdown due to inactivity...
2023-06-10 06:07:15,094 - INFO     - Executing shutdown...
2023-06-10 06:07:15,094 - ERROR    - Exception on /shutdown [GET]
Traceback (most recent call last):
  File "/home/daniel/dev/WHR/env/lib/python3.9/site-packages/flask/app.py", line 2529, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/daniel/dev/WHR/env/lib/python3.9/site-packages/flask/app.py", line 1825, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/daniel/dev/WHR/env/lib/python3.9/site-packages/flask/app.py", line 1823, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/daniel/dev/WHR/env/lib/python3.9/site-packages/flask/app.py", line 1799, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/home/daniel/dev/WHR/env/lib/python3.9/site-packages/dtale/app.py", line 438, in shutdown
    shutdown_server()
  File "/home/daniel

In [ ]:

Xs = Xc / Xc.std(axis=0, ddof=1)

np.max(np.abs(Xs[:,1] - (X[:,1] - X[:,1].mean())/X[:,1].std(ddof=1)))

In [ ]:
import sklearn.decomposition as decomp

fa = decomp.FactorAnalysis(n_components=100, rotation='varimax').fit(X)

In [ ]:
# dtale.show(pd.DataFrame(fa.components_.T, index=evenModel.model.coords['playerDay']))
fa.get_params()

In [ ]:
U, s, Vh = scipy.linalg.svd(Xs, full_matrices=False)

In [ ]:
s2 = np.square(s)

assert np.allclose(Xs[:,0] @ Xs[:,1], (s2 * Vh[:,0]) @ (Vh[:,1]))

s2norm = s2/(X.shape[0]-1)

In [ ]:
# a.T Xs.T Xs b = a.T Vh.T s s Vh b = (Vh a).T s s (Vh b)
pm.MvNormal()

# xh (Xh X)^-1 x
# = xh V S^-2 Vh x
# = xh V S^-2 Vh x
# = xh (V S^-1) (S^-1 Vh) x
# = xh (V S^-1) (V S^-1)h x

# X = U S Vh
# Xh X = V S² Vh = V Uh X Xh U Vh
# X Xh = U S² Uh

# (Xh X)/3999 - 1 = V (S²/3999 - 1) Vh

fig, ax = plt.subplots()
# ax.set_yscale('log')
# ax.plot(s2norm-1)
ax.plot((s2norm - 1).cumsum())
# x=np.arange(len(s2))
# ax.plot(x,1-1/(x/6000+1)**10)
pc0 = pd.Series(Vh[0], index=evenModel.model.coords['playerDay']).sort_values()
# pd.Series(Vh[3500], index=evenModel.model.coords['playerDay']).sort_values().plot()

def approxCorr(i, j, a):
    return ((s2[:a] * Vh[:a,i]) @ (Vh[:a,j]))/3999

approxs=np.linspace(1,4000,100,dtype=int)
# ax.plot(approxs, [[approxCorr(i,i,a) for i in np.linspace(0, X.shape[1]-1, 20, dtype='int')] for a in approxs])

Xs[:,0] @ Xs[:,0]

In [ ]:
Y = np.random.normal(size=(4000,5))
Ys = (Y - Y.mean(axis=0))/Y.std(axis=0,ddof=1)

In [ ]:
(Ys.T @ Ys)/3999 - np.eye(5)

In [ ]:
UY, SY, VhY = scipy.linalg.svd(Ys, full_matrices=False)

In [ ]:
# VhY.T @ np.diag(SY**2/3999 - 1) @ VhY
np.diag(SY**2/3999 - 1)